In [ ]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
# _set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

# Optional, add tracing in LangSmith
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "🚀Custom_OpenAI_Agent"



In [ ]:
from dataclasses import dataclass

@dataclass
class Person:
	name: str
	age: int

Person(name="Sam", age="10")

In [ ]:
Person(name="Sam", age="10").age + 1

In [ ]:
from pydantic import BaseModel

class Person(BaseModel):
	name: str
	age: int

Person(name="Sam", age="10")

In [ ]:
Person(name="Sam", age="10").age + 1

In [ ]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()


class PythonPackage(BaseModel):
    name: str
    author: str

resp = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": "Return the `name`, and `author` of pydantic, in a json object."
        },
    ]
)

PythonPackage.model_validate_json(resp.choices[0].message.content)

In [ ]:
import json

resp = client.chat.completions.create(
	model="gpt-4o",
	messages=[
		{
			"role": "user",
			"content": "Pydantic and FastAPI?",
		},
	],
	tools=[
		{
			"type": "function",
			"function": {
				"name": "Requirements",
				"description": "A list of packages and their first authors.",
				"parameters": {
					"$defs": {
						"Package": {
							"properties": {
								"name": {"title": "Name", "type": "string"},
								"author": {"title": "Author", "type": "string"},
							},
							"required": ["name", "author"],
							"title": "Package",
							"type": "object",
						}
					},
					"properties": {
						"packages": {
							"items": {"$ref": "#/$defs/Package"},
							"title": "Packages",
							"type": "array",
						}
					},
					"required": ["packages"],
					"title": "Packages",
					"type": "object",
				},
			},
		}
	],
	tool_choice={
		"type": "function",
		"function": {"name": "Requirements"},
	},
)

resp = json.loads(resp.choices[0].message.tool_calls[0].function.arguments)


In [ ]:
resp

In [1]:
from typing import List
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()


class PythonPackage(BaseModel):
    name: str
    author: str


class Packages(BaseModel):
    packages: List[PythonPackage]


resp = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": "Pydantic and FastAPI?",
        },
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "Requirements",
                "description": "A list of packages and their first authors.",
                "parameters": Packages.model_json_schema(),
            },
        }
    ],
    tool_choice={
        "type": "function",
        "function": {"name": "Requirements"},
    },
)

# Packages.model_validate_json(
#     resp.choices[0].message.tool_calls[0].function.arguments
# )


In [2]:
resp

ChatCompletion(id='chatcmpl-9g89PZQ9EQlzm6xlRyUn8cmbeHZXz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_DwXYaHovdiQNvcyP1QV4ABWA', function=Function(arguments='{"packages":[{"name":"Pydantic"},{"name":"FastAPI"}]}', name='Requirements'), type='function')]))], created=1719828135, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_4008e3b719', usage=CompletionUsage(completion_tokens=16, prompt_tokens=86, total_tokens=102))